In [13]:
import itertools
import lime_explainer as lime

In [2]:
feature_names = ['customer_id', 'monetary_dnn', 'monetary_btyd', 'frequency_dnn',
                 'frequency_btyd', 'recency', 'T', 'time_between',
                 'avg_basket_value', 'avg_basket_size', 'cnt_returns',
                 'has_returned', 'frequency_btyd_clipped', 'monetary_btyd_clipped',
                 'target_monetary_clipped', 'target_monetary']

In [3]:
explainer = lime.Explainer()

In [4]:
explainer.load_data(
  gcs_path = 'gs://ml-clv-data/model_data.csv',
  features_to_use = [1,3,5,6,7,8,9,10,11],
  feature_names = feature_names,
  target_idx = 15,
  skip_first=True,
  integer_rows=[1,2,3,7,8]
)

In [5]:
explainer.create_cmle_client(
  gcp_project = 'ml-clv',
  gcp_model = 'dnn_airflow',
  gcp_model_version = 'v1',
  output_func = lambda x : x['predicted_monetary'] 
)

In [6]:
record, df = explainer.explain_random_record()

Intercept 11038.618081542227
Prediction_local [9964.70413089]
Right: 6469.44140625


In [7]:
print(record)

[5335.55, 93.0, 332.0, 333.0, 3.57, 57.37, 17.9, 0.0, 0.0]


In [8]:
df

,representation,weight
0,monetary_dnn > 4087.26,16496.327771
1,recency > 331.00,-9440.772419
2,331.00 < T <= 334.00,-7536.684959
3,time_between <= 3.67,-4248.530095
4,has_returned <= 0.00,3655.745752


In [9]:
mod_record = ','.join(['2000'] + [str(e) for e in record[1:]])
print('Mod Record: {}\n\n'.format(mod_record))
_, mod_df = explainer.explain_record(mod_record)

Mod Record: 2000,93.0,332.0,333.0,3.57,57.37,17.9,0.0,0.0


Intercept 2494.599610660366
Prediction_local [2009.02564277]
Right: 1795.589599609375


In [10]:
mod_df

,representation,weight
0,331.00 < T <= 334.00,-1100.115940
1,44.28 < avg_basket_value <= 66.77,1035.075662
2,recency > 331.00,640.618780
3,1167.98 < monetary_dnn <= 2423.27,-559.371108
4,cnt_returns <= 0.00,-501.781362


In [14]:
text_explanations = explainer.explain_model()

Intercept 2859.7952306099787
Prediction_local [2062.08399]
Right: 2603.36328125
Intercept 7393.896245742169
Prediction_local [25854.30042216]
Right: 7457.65478515625
Intercept 5652.968508861921
Prediction_local [843.32090619]
Right: -469.25927734375
Intercept 6287.8629423421535
Prediction_local [4908.61255091]
Right: 497.15673828125
Intercept 3709.5847428260413
Prediction_local [3591.93857192]
Right: 3507.704345703125


In [15]:
text_explanations

[[('frequency_dnn > 83.00', 20175.19405858082),
  ('recency > 331.00', -16796.47249640443),
  ('T > 334.00', 11735.970951304182),
  ('28.37 < avg_basket_value <= 44.28', 4945.884803533934),
  ('cnt_returns <= 0.00', -1600.1731405979951)],
 [('monetary_dnn <= 1167.98', -3995.6483292516373),
  ('9.17 < avg_basket_size <= 14.75', -853.7516528778838),
  ('T <= 306.00', 831.4782318299176),
  ('has_returned <= 0.00', -516.833502895875),
  ('recency <= 291.00', -274.8923494715958)],
 [('T <= 306.00', 1548.8694092175406),
  ('9.17 < avg_basket_size <= 14.75', -1367.9349385875644),
  ('291.00 < recency <= 325.00', -1081.753515292896),
  ('5.49 < time_between <= 8.22', 708.3515795457837),
  ('1167.98 < monetary_dnn <= 2423.27', -605.2437754950245)],
 [('9.17 < avg_basket_size <= 14.75', 9830.113922440198),
  ('time_between > 8.22', -5571.300765861717),
  ('T > 334.00', -5254.038365014957),
  ('monetary_dnn <= 1167.98', -4143.835425143651),
  ('cnt_returns <= 0.00', 3759.810242145079)],
 [('9.17 